In [2]:
import pandas as pd
import db2 as db

import json as js

path_meta_db = db.path_meta_db
path_meta_dl = db.path_meta_dl

columns_db = ['name', 'path', 'nlayer', 'nE', 'nnu', 'nborder', 'type', 'ndim', 'nnode', 'ndb', 'ntrain', 'nval', 'ntest', 'nrand', 'aE', 'bE', 'anu', 'bnu', 'aborder', 'bborder', 'fE', 'fnu', 'fborder']
columns_dl = ['name', 'path', 'name_db', 'x', 'y', 'batch', 'epochs', 'layers', 'last_activation', 'loss', 'metrics']

df_db = pd.read_csv(path_meta_db)
df_dl = pd.read_csv(path_meta_dl)

def check_dict_db(dict_db):

    requied_keys  = ('name', 'type', 'nnode', 'nval', 'ntest', 'aE',   'bE',  'anu', 'bnu', 'aborder', 'bborder')
    requied_key_types = ( str,    str,    int,     int,    int,     tuple,  tuple, tuple, tuple, tuple,     tuple)
    ab_key = ['aE', 'bE', 'anu', 'bnu', 'aborder', 'bborder']
    aa_key = [ 0.1,  0.1,  0.1,   0.1,   0.05,      0.05]
    bb_key = [ 10.,  10.,  0.5,   0.5,   1.0,       1.0]
    border_delay = 0.05
    requied_types = ['FF', 'LH']

    for key in requied_keys:
        if key not in dict_db.keys():
            return 'Нет ключа ' + key
    
    for key in dict_db.keys():
        if key not in  requied_keys:
            return 'Лишний ключ ' + key
        
    for i, key in enumerate(requied_keys):
        if type(dict_db[key]) is not requied_key_types[i]:
            print(type(dict_db[key]), requied_key_types[i])
            return 'Неверный тип у ' + key
    
    if len(dict_db['aE']) < 2:
        return 'Длина aE меньше двух: ' + len(dict_db['aE'])
    
    for key in ab_key:
        if len(dict_db['aE']) != len(dict_db[key]):
            return 'Длина aE не равна длине ' + key + ': ' + str(len(dict_db['aE'])) + ' ' + str(len(dict_db[key]))
        for i in range(len(dict_db[key])):
            if type(dict_db[key][i]) is not float:
                return 'Неверный тип в ' + key + ': ' + str(dict_db[key])
    
    #************************************************
    
    if dict_db['nnode'] <= 1:
        return 'Не верный nnode: ' + str(dict_db['nnode'])
    
    if dict_db['nval'] <= 0:
        return 'Не верный nval: ' + str(dict_db['nval'])
    
    if dict_db['ntest'] <= 0:
        return 'Не верный ntest: ' + str(dict_db['ntest'])
    
    #************************************************

    for i, key in enumerate(ab_key):
        for elem in dict_db[key]:
            if elem < aa_key[i] or elem > bb_key[i]:
                return 'Выход за границы ' + key + ': ' + str(dict_db[key])
    
    ndim = 0

    for i in range(0, len(ab_key), 2):
        for j in range(len(dict_db[ab_key[0]])):
            if dict_db[ab_key[i]][j] > dict_db[ab_key[i + 1]][j]:
                return 'a больше b у ' + ab_key[i] + ': ' + str(dict_db[ab_key[i]][j]) + ' ' + str(dict_db[ab_key[i + 1]][j])
            if dict_db[ab_key[i]][j] < dict_db[ab_key[i + 1]][j]:
                ndim += 1
    
    for key in ab_key[-2:]:
        for i in range(len(dict_db[key]) - 1):
            if dict_db[key][i + 1] - dict_db[key][i] < border_delay:
                return 'Нарушение растояния между границами ' + key + ': ' + str(dict_db[key])
        if dict_db[key][-1] != 1.0:
            return 'Последняя граница не 1.0 у ' + key + ': ' + str(dict_db[key])


    #************************************************
            
    if dict_db['type'] not in requied_types:
        return 'Не верный тип: ' + str(dict_db['type'])
    
    ndb = 0
    if   dict_db['type'] == 'FF':
        ndb = dict_db['nnode']**ndim + dict_db['nval'] + dict_db['ntest']
        
    elif dict_db['type'] == 'LH':
        ndb = dict_db['nnode']       + dict_db['nval'] + dict_db['ntest']

    if ndb > 200000:
        return 'Избыток данных для обучения: ' + str(ndb)
    
    if dict_db['name'] in list(df_db['name']):
        return 'База данных с таким же именем уже существует:' + str(dict_db['name'])
    
    return 0

def add_db(dict_db):

    check = check_dict_db(dict_db)

    if check != 0:
        return check

    #************************************************

    ab_key = ['aE', 'bE', 'anu', 'bnu', 'aborder', 'bborder']

    ndim = 0
    
    for i in range(0, len(ab_key), 2):
        nkey = 0
        fkey = ()
        for j in range(len(dict_db[ab_key[0]])):
            if dict_db[ab_key[i]][j] < dict_db[ab_key[i + 1]][j]:
                ndim += 1
                nkey += 1
                fkey += (1,)
            else:
                fkey += (0,)
        dict_db['n' + ab_key[i][1:]] = nkey
        dict_db['f' + ab_key[i][1:]] = fkey
    
    dict_db['ndim'] = ndim

    #************************************************
    
    dict_db['nlayer'] = len(dict_db['aE'])

    if  dict_db['type'] == 'FF':
        dict_db['ntrain'] = dict_db['nnode']**dict_db['ndim']
    elif dict_db['type'] == 'LH':
        dict_db['ntrain'] = dict_db['nnode']

    
    dict_db['nrand'] = dict_db['nval']  + dict_db['ntest']
    dict_db['ndb']   = dict_db['nrand'] + dict_db['ntrain']

    dict_db['path'] = r'/nlayer_' + str(dict_db['nlayer']) + r'/ndim_' + str(dict_db['ndim']) + r'/' + dict_db['type'] + r'/' + dict_db['name']

    for key in dict_db.keys():
        if type(dict_db[key]) in [list, tuple]:
            dict_db[key] = js.dumps(dict_db[key])

    df_db.loc[len(df_db.index)] = dict_db

    return 'Новые данные о базе данных добавлены'

def check_dict_dl(dict_dl):

    requied_keys  =       ['name', 'name_db', 'x',   'y',   'batch', 'epochs', 'layers', 'last_activation', 'loss', 'metrics']
    requied_key_types =   [ str,    str,       tuple, tuple, int,     int,      tuple,    str,               str,    list]
    requied_losses =      ['msle', 'mse', 'mae', 'mape']
    requied_activations = ['relu', 'tanh', 'ctanh', 'linear']
    requied_x =           ['nref', 'tref', 'ref', 'nrvo', 'nrar']
    requied_y =           ['E1', 'E2', 'E3', 'E', 'nu1', 'nu2', 'nu3', 'nu', 'nE', 'tE']

    for key in requied_keys:
        if key not in dict_dl.keys():
            return 'Нет ключа ' + key
    
    for key in dict_dl.keys():
        if key not in  requied_keys:
            return 'Лишний ключ ' + key
        
    for i, key in enumerate(requied_keys):
        if type(dict_dl[key]) is not requied_key_types[i]:
            return 'Неверный тип у: ' + key
        
    if len(dict_dl['layers']) == 0:
        return 'Пустой массив layer: ' + str(dict_dl['layers'])
    
    for layer in dict_dl['layers']:
        if type(layer) is not tuple:
            return 'Неверный тип layer: ' + str(layer)
        
        if len(layer) != 2:
            return 'Неверная длина массива layer: ' + str(layer)
        
        if type(layer[0]) is not int:
            return 'Неверный тип layer[0]: ' + str(layer)
        
        if type(layer[1]) is not str:
            return 'Неверный тип layer[1]: ' + str(layer)
        
    if len(dict_dl['x']) == 0:
        return 'Пустой массив x: ' + str(dict_dl['x'])
    
    for x in dict_dl['x']:
        if type(x) is not tuple:
            return 'Неверный тип x: ' + str(x)
        
        if len(x) == 0:
            return 'Пустой массив x: ' + str(x)
    
        for xi in x:
            if type(xi) is not str:
                return 'Неверный тип x: ' + str(x)
    
    if len(dict_dl['y']) == 0:
        return 'Пустой массив y: ' + str(dict_dl['y'])
    
    for y in dict_dl['y']:
        if type(y) is not tuple:
            return 'Неверный тип y: ' + str(y)
        
        if len(y) == 0:
            return 'Пустой массив y: ' + str(y)
        
        for yi in y:
            if type(yi) is not str:
                return 'Неверный тип y: ' + str(y)
            
    for metric in dict_dl['metrics']:
        if type(metric) is not str:
            return 'Неверный тип metrics: ' + str(metric)
    
    #************************************************
        
    if dict_dl['batch'] <= 0:
        return 'Неверный batch: ' + str(dict_dl['batch'])
    
    if dict_dl['epochs'] <= 0:
        return 'Неверный epochs: ' + str(dict_dl['epochs'])
    
    if dict_dl['loss'] not in requied_losses:
        return 'Неверный loss: ' + str(dict_dl['loss'])
    
    if dict_dl['last_activation'] not in requied_activations:
        return 'Неверный last_activation: ' + dict_dl['last_activation']
    
    for x in dict_dl['x']:
        for xi in x:
            if xi not in requied_x:
                return 'Неверный x: ' + str(x)
    
    for y in dict_dl['y']:
        for yi in y:
            if yi not in requied_y:
                return 'Неверный y: ' + str(y)
    
    for metr in dict_dl['metrics']:
        if metr not in  requied_losses:
            return 'Неверная метрика: ' + metr
        
    for layer in dict_dl['layers']:
        if layer[0] <= 1:
            return 'Неверное количество нейронов: ' + str(layer)
        
        if layer[1] not in requied_activations:
            return 'Неверная активация: ' + str(layer)
        
    if dict_dl['epochs'] > 10**8:
        return 'Слишком большое количество эпох: ' + str(dict_dl['epochs'])
    
    #************************************************

    tmp_df = df_db.loc[df_db['name'] == dict_dl['name_db']].to_dict(orient='records')

    if len(tmp_df) == 0:
        return 'Нет базы данных с именем: ' + dict_dl['name_db']
    elif len(tmp_df) != 1:
        return 'ОШИБКА! Две базы данных с одинаковыми именами!' + dict_dl['name_db']
    
    tmp_df = tmp_df[0]

    ntrain = tmp_df['ntrain']

    if dict_dl['batch'] > ntrain:
        return 'batch больше ntrain: ' + str(dict_dl['batch']) + ' ' + str(ntrain)
    
    neurons = 0

    for i in range(len(dict_dl['layers']) - 1):
        neurons += dict_dl['layers'][i][0] * dict_dl['layers'][i + 1][0]
        
    # if neurons > ntrain:
    #     return 'Нейроннов в сети больше, чем ntrain ' + str(neurons) + ' ' + str(ntrain)
    
    #************************************************

    if dict_dl['name'] in list(df_dl['name']):
        return 'База данных с таким же именем уже существует:' + str(dict_dl['name'])

    return 0

def add_dl(dict_dl):
    
    check = check_dict_dl(dict_dl)

    if check != 0:
        return check
    
    dict_dl['path'] = r'/models/' + dict_dl['name']

    for key in dict_dl.keys():
        if type(dict_dl[key]) in [list, tuple]:
            dict_dl[key] = js.dumps(dict_dl[key])
    
    df_dl.loc[len(df_dl.index)] = dict_dl
    
    return 'Новые данные о базе данных добавлены' 

In [77]:
new_db = {
    'name':    'LH100_0520_0104',
    'type':    'LH',
    'nnode':   100000,
    'nval':    10000,
    'ntest':   20000,

    'aE':      (0.5, 0.5, 0.5),
    'bE':      (2.0, 2.0, 2.0),

    'anu':     (0.1, 0.1, 0.1),
    'bnu':     (0.4, 0.4, 0.4),

    'aborder': (0.33, 0.66, 1.0),
    'bborder': (0.33, 0.66, 1.0)
}
print(add_db(new_db))
df_db[['name', 'path', 'type', 'nnode', 'ntrain', 'nval', 'ntest', 'aE', 'bE', 'anu', 'bnu', 'aborder', 'bborder']]

База данных с таким же именем уже существует:LH100_0520_0104


,name,path,type,nnode,ntrain,nval,ntest,aE,bE,anu,bnu,aborder,bborder
0,FF16_0520,/nlayer_3/ndim_3/FF/FF16_0520,FF,16,4096,1000,10000,"[0.5, 0.5, 0.5]","[2.0, 2.0, 2.0]","[0.3, 0.3, 0.3]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
1,FF16_0250,/nlayer_3/ndim_3/FF/FF16_0250,FF,16,4096,1000,10000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.3, 0.3, 0.3]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
2,LH4000_0250,/nlayer_3/ndim_3/LH/LH4000_0250,LH,4000,4000,1000,10000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.3, 0.3, 0.3]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
3,LH10000_0250_0304,/nlayer_3/ndim_6/LH/LH10000_0250_0304,LH,10000,10000,1000,10000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.3, 0.3, 0.3]","[0.4, 0.4, 0.4]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
4,LH10000_0250_0203,/nlayer_3/ndim_6/LH/LH10000_0250_0203,LH,10000,10000,1000,10000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.2, 0.2, 0.2]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
5,LH4000_0520,/nlayer_3/ndim_3/LH/LH4000_0520,LH,4000,4000,1000,10000,"[0.5, 0.5, 0.5]","[2.0, 2.0, 2.0]","[0.3, 0.3, 0.3]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
6,LH12000_0250,/nlayer_3/ndim_3/LH/LH12000_0250,LH,12000,12000,4000,15000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.3, 0.3, 0.3]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
7,LH20_0250,/nlayer_3/ndim_3/LH/LH20_0250,LH,20000,20000,4000,15000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.3, 0.3, 0.3]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
8,LH30_0250,/nlayer_3/ndim_3/LH/LH30_0250,LH,30000,30000,10000,15000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.3, 0.3, 0.3]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
9,LH20_0520_0304,/nlayer_3/ndim_6/LH/LH20_0520_0304,LH,20000,20000,10000,15000,"[0.5, 0.5, 0.5]","[2.0, 2.0, 2.0]","[0.3, 0.3, 0.3]","[0.4, 0.4, 0.4]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"


In [108]:
new_dl = {
    'name':    'mod_LH30_0520_0304_100_MAPE_batch13_3in',
    'name_db': 'LH30_0520_0304',
    'x':       (('nref', 'nrvo', 'nrar'),),
    'y':       (('E', 'nu'),),
    'batch':   2**13,
    'epochs':  10**8,
    'layers':  (
        (100, 'tanh'),
        (100, 'tanh'),
        (100, 'tanh'),
        (100, 'tanh'),
        (100, 'tanh'),
        (100, 'tanh'),
        (100, 'tanh')
    ),
    'last_activation': 'linear',
    'loss':            'mape',
    'metrics':        ['mape']
}
print(add_dl(new_dl))
df_dl[['name', 'name_db', 'x', 'y', 'batch', 'epochs', 'layers', 'loss']][-10:]

Новые данные о базе данных добавлены


,name,name_db,x,y,batch,epochs,layers,loss
57,mod_LH100_0520_0304_01nu,LH100_0520_0304,"[[""nref""]]","[[""nu""]]",8192,100000000,"[[30, ""tanh""], [30, ""tanh""], [30, ""tanh""], [30...",mape
58,mod_LH100_0520_0104_01E,LH100_0520_0104,"[[""nref""]]","[[""E""]]",8192,100000000,"[[30, ""tanh""], [30, ""tanh""], [30, ""tanh""], [30...",mape
59,mod_LH30_0520_0304_01_MAPE,LH30_0520_0304,"[[""nref""]]","[[""E"", ""nu""]]",256,100000000,"[[20, ""tanh""], [20, ""tanh""], [20, ""tanh""], [20...",mape
60,mod_LH30_0520_0304_01_MAPE_batch2,LH30_0520_0304,"[[""nref""]]","[[""E"", ""nu""]]",8192,100000000,"[[20, ""tanh""], [20, ""tanh""], [20, ""tanh""], [20...",mape
61,mod_LH30_0520_0304_01E_MAPE_batch2,LH30_0520_0304,"[[""nref""]]","[[""E""]]",8192,100000000,"[[20, ""tanh""], [20, ""tanh""], [20, ""tanh""], [20...",mape
62,mod_LH30_0520_0304_01E1_MAPE_batch2,LH30_0520_0304,"[[""nref""]]","[[""E1""]]",8192,100000000,"[[20, ""tanh""], [20, ""tanh""], [20, ""tanh""], [20...",mape
63,mod_LH30_0520_0304_100_MAPE_batch13,LH30_0520_0304,"[[""nref""]]","[[""E"", ""nu""]]",8192,100000000,"[[100, ""tanh""], [100, ""tanh""], [100, ""tanh""], ...",mape
64,mod_LH30_0520_0304_100_MAPE_batch13_relu,LH30_0520_0304,"[[""nref""]]","[[""E"", ""nu""]]",8192,100000000,"[[100, ""relu""], [100, ""relu""], [100, ""relu""], ...",mape
65,mod_LH30_0520_0304_20_MAPE_batch13_3in,LH30_0520_0304,"[[""nref"", ""nrvo"", ""nrar""]]","[[""E"", ""nu""]]",8192,100000000,"[[20, ""tanh""], [20, ""tanh""], [20, ""tanh""], [20...",mape
66,mod_LH30_0520_0304_100_MAPE_batch13_3in,LH30_0520_0304,"[[""nref"", ""nrvo"", ""nrar""]]","[[""E"", ""nu""]]",8192,100000000,"[[100, ""tanh""], [100, ""tanh""], [100, ""tanh""], ...",mape


In [59]:
df_db.to_csv(path_meta_db, index=False)

In [109]:
df_dl.to_csv(path_meta_dl, index=False)

In [26]:
df_db.loc[df_db['ndim'] == 6][['name', 'path', 'type', 'nnode', 'ntrain', 'nval', 'ntest', 'aE', 'bE', 'anu', 'bnu', 'aborder', 'bborder']]

,name,path,type,nnode,ntrain,nval,ntest,aE,bE,anu,bnu,aborder,bborder
3,LH10000_0250_0304,/nlayer_3/ndim_6/LH/LH10000_0250_0304,LH,10000,10000,1000,10000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.3, 0.3, 0.3]","[0.4, 0.4, 0.4]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
4,LH10000_0250_0203,/nlayer_3/ndim_6/LH/LH10000_0250_0203,LH,10000,10000,1000,10000,"[0.2, 0.2, 0.2]","[5.0, 5.0, 5.0]","[0.2, 0.2, 0.2]","[0.3, 0.3, 0.3]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
9,LH20_0520_0304,/nlayer_3/ndim_6/LH/LH20_0520_0304,LH,20000,20000,10000,15000,"[0.5, 0.5, 0.5]","[2.0, 2.0, 2.0]","[0.3, 0.3, 0.3]","[0.4, 0.4, 0.4]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
10,LH30_0520_0304,/nlayer_3/ndim_6/LH/LH30_0520_0304,LH,30000,30000,10000,15000,"[0.5, 0.5, 0.5]","[2.0, 2.0, 2.0]","[0.3, 0.3, 0.3]","[0.4, 0.4, 0.4]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
11,LH50_0520_0104,/nlayer_3/ndim_6/LH/LH50_0520_0104,LH,50000,50000,10000,20000,"[0.5, 0.5, 0.5]","[2.0, 2.0, 2.0]","[0.1, 0.1, 0.1]","[0.4, 0.4, 0.4]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"
12,LH60_0520_0304,/nlayer_3/ndim_6/LH/LH60_0520_0304,LH,60000,60000,10000,20000,"[0.5, 0.5, 0.5]","[2.0, 2.0, 2.0]","[0.3, 0.3, 0.3]","[0.4, 0.4, 0.4]","[0.33, 0.66, 1.0]","[0.33, 0.66, 1.0]"


In [4]:
df_dl.loc[df_dl['name_db'] == 'FF16_0250']

,name,path,name_db,x,y,batch,epochs,layers,last_activation,loss,metrics
0,mod_FF_0250_01,/models/mod_FF_0250_01,FF16_0250,"[[""nref""]]","[[""E1"", ""E2"", ""E3""]]",256,1000000,"[[17, ""tanh""], [17, ""tanh""], [17, ""tanh""], [17...",linear,msle,"[""mse"", ""mape""]"
55,mod_FF16_0250_MAPE,/models/mod_FF16_0250_MAPE,FF16_0250,"[[""nref""]]","[[""E""]]",256,100000000,"[[17, ""tanh""], [17, ""tanh""], [17, ""tanh""], [17...",linear,mape,"[""mape""]"


In [81]:
df_dl = df_dl.drop(61)

In [21]:
df_dl.iloc[20]

name                                           mod_LH30_0520_0304_03
path                                   /models/mod_LH30_0520_0304_03
name_db                                               LH30_0520_0304
x                                                         [["nref"]]
y                                                      [["E", "nu"]]
batch                                                            256
epochs                                                       1000000
layers             [[22, "tanh"], [22, "tanh"], [22, "tanh"], [22...
last_activation                                               linear
loss                                                            msle
metrics                                                     ["mape"]
Name: 20, dtype: object